# Char-RNN stateless
* This RNN generate char by char sentences trained by Shakespeare's work
* this RNN is statefull, so the state at eache timestep is preserved.

## Import dependencies

In [83]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

## Creating Dataset

In [84]:
#Download data
shakespeare_url = "http://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as file:
    shakespeare_text = file.read()

In [85]:
# Tokenize chars
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)  # char_level=True allows char 
                                                                 # encoding instead default word enoding
tokenizer.fit_on_texts(shakespeare_text)

In [86]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1  # [encoded] = encoded.squeeze()

In [87]:
# Split Sequential dataset
dataset_size = tokenizer.document_count
train_size = dataset_size * 90 // 100

In [88]:
n_steps = 100
window_length = n_steps + 1
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [89]:
model = keras.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=.2, 
                     recurrent_dropout=.2, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=.2, recurrent_dropout=.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [90]:
# Callback class to reset state in each epoch
class ResetStateCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [97]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, epochs=50, steps_per_epoch=steps_per_epoch, callbacks=[ResetStateCallback()])

Epoch 1/50
313/313 [==============================] - 55s 175ms/step - loss: 1.9893
Epoch 2/50
313/313 [==============================] - 60s 192ms/step - loss: 2.5787
Epoch 3/50
313/313 [==============================] - 61s 194ms/step - loss: 2.1947
Epoch 4/50
313/313 [==============================] - 61s 195ms/step - loss: 1.9874
Epoch 5/50
313/313 [==============================] - 63s 202ms/step - loss: 1.9182
Epoch 6/50
313/313 [==============================] - 66s 211ms/step - loss: 1.7921
Epoch 7/50
313/313 [==============================] - 96s 306ms/step - loss: 1.7622
Epoch 8/50
313/313 [==============================] - 101s 322ms/step - loss: 1.7386
Epoch 9/50
313/313 [==============================] - 101s 322ms/step - loss: 1.7215
Epoch 10/50
313/313 [==============================] - 101s 323ms/step - loss: 1.7053
Epoch 11/50
313/313 [==============================] - 83s 266ms/step - loss: 1.6925
Epoch 12/50
313/313 [==============================] - 60s 190ms/step -

In [103]:
def process(text):
    X = np.array(tokenizer.texts_to_sequences(text)) - 1
    return tf.one_hot(X, max_id)

In [113]:
def next_char(text, temperature=1):
    X_new = process([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [114]:
def complete_text(text, n_char=50, temperature=1):
    for _ in range(n_char):
        text += next_char(text, temperature)
    return text

In [115]:
print(complete_text("i", temperature=.8))

InvalidArgumentError:  Specified a list with shape [32,39] from a tensor with shape [1,39]
	 [[node sequential_1/gru_2/TensorArrayUnstack/TensorListFromTensor (defined at <ipython-input-110-6a9a2e44a30f>:3) ]] [Op:__inference_predict_function_50756]

Function call stack:
predict_function
